In [1]:
from bids import BIDSLayout

In [2]:
layout = BIDSLayout('/data/risk_precision/garcia_ruff_risk_precision_fmri/', validate=False)

In [3]:
df = layout.to_df()

In [4]:
df.subject.unique()

array([nan, '11927', '19131', '20466', '21246', '24613', '24626', '24933',
       '25427', '26753', '26867', '26905', '28051', '30130', '30451',
       '30733', '31422', '31724', '31814', '32091', '32829', '33156',
       '33464', '33524', '34501', '34626', '35091', '35092', '35114',
       '35177', '35202', '35721', '35982', '36874', '37073', '37257',
       '37268', '37305', '37368', '37982', '38312', '38422', '38629',
       '38726', '38940', '38972', '39010', '39051', '39089', '39115',
       '39152', '39166', '39180', '39203', '39283', '39509', '39572',
       '39575', '39627', '39693', '39903', '39960', '39999', '40048',
       '40359', '40425'], dtype=object)

In [5]:
df = df[~df.subject.isnull()]

In [25]:
import re
import json
metadata = df[df.suffix == 'sessions']

reg = re.compile('SNS_RPREC_(?P<subject_id>[X0-9]+)_[0-9]+')

def get_original_subject_id(row):
    with open(row.path) as f:
        d = json.load(f)[0]
        return reg.match(d['MRI_KEY']).group(1)
    
ori_ids = metadata.apply(get_original_subject_id, 1)
ori_ids[ori_ids == '21X'] = 21
ori_ids = ori_ids.astype(int).to_frame('id')
ori_ids = ori_ids.set_index(metadata.subject)

In [28]:
ori_ids[ori_ids.id == 21]

,id
subject,
37982,21


In [30]:
t1w = df[(df.suffix == 'T1w') & (df.extension != 'json')]

In [32]:
import nibabel as nb

In [34]:
t1w['n_slices'] = t1w.path.apply(lambda t: nb.load(t).shape[-1])

/Users/gdehol/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [38]:
tmp = t1w[t1w.n_slices ==170].groupby(['subject']).size()
tmp[tmp > 1]

subject
32829    2
dtype: int64

In [39]:
t1w = t1w[t1w.n_slices == 170]

In [42]:
import os
import os.path as op

In [48]:
import shutil

In [59]:
row.extension

'nii.gz'

In [63]:
import gzip

In [62]:
for ix, row in t1w.iterrows():
    
    if row.subject != '32829':
        
        ori_id = ori_ids.loc[row.subject].id

        base_dir = '/data/risk_precision/ds-risk/sourcedata/sub-{}/anat'.format(ori_id)
        if not op.exists(base_dir):
            os.makedirs(base_dir)

        
        t1w_fn = op.join(base_dir, 'sub-{}_acq-{}_T1w.nii.gz').format(ori_id, row.acquisition)
        
        if row.extension == 'nii':
            print('compressing {} -> {}'.format(row.path, t1w_fn))
            with open(row.path, 'rb') as f_in:
                with gzip.open(t1w_fn, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
        elif row.extension == 'nii.gz':
            shutil.copy(row.path, t1w_fn)
        else:
            raise Exception('wut?')
        
        


compressing /data/risk_precision/garcia_ruff_risk_precision_fmri/sub-19131/ses-21869/anat/sub-19131_ses-21869_acq-11_rec-1_T1w.nii -> /data/risk_precision/ds-risk/sourcedata/sub-3/anat/sub-3_acq-11_T1w.nii.gz
compressing /data/risk_precision/garcia_ruff_risk_precision_fmri/sub-24613/ses-21916/anat/sub-24613_ses-21916_acq-11_rec-1_T1w.nii -> /data/risk_precision/ds-risk/sourcedata/sub-2/anat/sub-2_acq-11_T1w.nii.gz
compressing /data/risk_precision/garcia_ruff_risk_precision_fmri/sub-33524/ses-21702/anat/sub-33524_ses-21702_acq-11_rec-1_T1w.nii -> /data/risk_precision/ds-risk/sourcedata/sub-0/anat/sub-0_acq-11_T1w.nii.gz
compressing /data/risk_precision/garcia_ruff_risk_precision_fmri/sub-35721/ses-21870/anat/sub-35721_ses-21870_acq-11_rec-1_T1w.nii -> /data/risk_precision/ds-risk/sourcedata/sub-4/anat/sub-4_acq-11_T1w.nii.gz
compressing /data/risk_precision/garcia_ruff_risk_precision_fmri/sub-38422/ses-21867/anat/sub-38422_ses-21867_acq-11_rec-1_T1w.nii -> /data/risk_precision/ds-risk/s

In [64]:
bold = df[(df.suffix == 'bold') & (df.extension != 'json')]

In [67]:
for ix, row in bold.iterrows():
    
    if row.subject != '32829':
        
        ori_id = ori_ids.loc[row.subject].id

        base_dir = '/data/risk_precision/ds-risk/sourcedata/sub-{}/func'.format(ori_id)
        if not op.exists(base_dir):
            os.makedirs(base_dir)
        
        bold_fn = op.join(base_dir, 'sub-{}_task-numrisk_acq-{}_run-{}_bold.nii.gz').format(ori_id, row.acquisition, row.run)
        
        if row.extension == 'nii':
            print('compressing {} -> {}'.format(row.path, bold_fn))
            with open(row.path, 'rb') as f_in:
                with gzip.open(bold_fn, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
        elif row.extension == 'nii.gz':
            shutil.copy(row.path, bold_fn)
        else:
            raise Exception('wut?')

compressing /data/risk_precision/garcia_ruff_risk_precision_fmri/sub-19131/ses-21869/func/sub-19131_ses-21869_task-fmri_acq-3_rec-1_run-1_bold.nii -> /data/risk_precision/ds-risk/sourcedata/sub-3/func/sub-3_task-numrisk_acq-3_run-1_bold.nii.gz
compressing /data/risk_precision/garcia_ruff_risk_precision_fmri/sub-19131/ses-21869/func/sub-19131_ses-21869_task-fmri_acq-4_rec-1_run-2_bold.nii -> /data/risk_precision/ds-risk/sourcedata/sub-3/func/sub-3_task-numrisk_acq-4_run-2_bold.nii.gz
compressing /data/risk_precision/garcia_ruff_risk_precision_fmri/sub-19131/ses-21869/func/sub-19131_ses-21869_task-fmri_acq-5_rec-1_run-3_bold.nii -> /data/risk_precision/ds-risk/sourcedata/sub-3/func/sub-3_task-numrisk_acq-5_run-3_bold.nii.gz
compressing /data/risk_precision/garcia_ruff_risk_precision_fmri/sub-19131/ses-21869/func/sub-19131_ses-21869_task-fmri_acq-7_rec-1_run-4_bold.nii -> /data/risk_precision/ds-risk/sourcedata/sub-3/func/sub-3_task-numrisk_acq-7_run-4_bold.nii.gz
compressing /data/risk_p